In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e12/sample_submission.csv
/kaggle/input/playground-series-s5e12/train.csv
/kaggle/input/playground-series-s5e12/test.csv


In [2]:
# Agentic Kaggle Pipeline — Full Code (cleaned & corrected)
# Paths assume /kaggle/input/playground-series-s5e12/
# Requirements (Kaggle kernel has these): pandas, numpy, scikit-learn, lightgbm

import os
import gc
import json
import numpy as np
import pandas as pd
from typing import Tuple, Dict, Any, List

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

# -----------------------
# 0) Paths & Target
# -----------------------
INPUT_DIR = "/kaggle/input/playground-series-s5e12"
TRAIN_PATH = os.path.join(INPUT_DIR, "/kaggle/input/playground-series-s5e12/train.csv")
TEST_PATH  = os.path.join(INPUT_DIR, "/kaggle/input/playground-series-s5e12/train.csv")
SAMPLE_PATH = os.path.join(INPUT_DIR, "/kaggle/input/playground-series-s5e12/train.csv")

TARGET = "diagnosed_diabetes"
ID = "id"
N_SPLITS = 5
RANDOM_STATE = 42

# -----------------------
# 1) Load data
# -----------------------
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sample = pd.read_csv(SAMPLE_PATH)

print("train shape:", train.shape)
print("test shape: ", test.shape)

# -----------------------
# 2) Basic preprocessing
# -----------------------
# Build feature list (exclude id, target)
features = [c for c in train.columns if c not in [ID, TARGET]]
print(f"{len(features)} features detected.")

# Detect object cols and label-encode (safe)
obj_cols = [c for c in features if train[c].dtype == "object"]
print("Categorical cols:", obj_cols)

for col in obj_cols:
    le = LabelEncoder()
    combined = pd.concat([train[col].fillna("##NA##").astype(str), test[col].fillna("##NA##").astype(str)], axis=0)
    le.fit(combined)
    train[col] = le.transform(train[col].fillna("##NA##").astype(str))
    test[col]  = le.transform(test[col].fillna("##NA##").astype(str))

# Numeric fill: median (no chained assignment)
num_cols = [c for c in features if c not in obj_cols]
for col in num_cols:
    med = train[col].median()
    train[col] = train[col].fillna(med)
    test[col]  = test[col].fillna(med)

# Add small, plausible engineered features only
if "bmi" in train.columns and "age" in train.columns:
    train["bmi_x_age"] = train["bmi"] * train["age"]
    test["bmi_x_age"]  = test["bmi"] * test["age"]
    if "bmi_x_age" not in features:
        features.append("bmi_x_age")

# Missing-count signal (defensive)
train["n_missing"] = train[features].isna().sum(axis=1)
test["n_missing"]  = test[features].isna().sum(axis=1)
if "n_missing" not in features:
    features.append("n_missing")

print("Target positive ratio:", train[TARGET].mean())

# -----------------------
# 3) Training function
# -----------------------
def train_model_and_return_outputs(X: pd.DataFrame = None,
                                   y: np.ndarray = None,
                                   X_test: pd.DataFrame = None,
                                   features_list: List[str] = None,
                                   params: dict = None,
                                   n_splits: int = N_SPLITS,
                                   seed: int = RANDOM_STATE) -> Tuple[np.ndarray, np.ndarray, pd.DataFrame]:
    """
    Train LightGBM with Stratified K-Fold CV using callback-based early stopping.
    Returns: oof_preds, test_preds, feature_importance_df
    """
    if features_list is None:
        features_list = features.copy()
    if X is None:
        X = train[features_list].copy()
    if y is None:
        y = train[TARGET].values
    if X_test is None:
        X_test = test[features_list].copy()

    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(X_test))
    feature_importance_df = pd.DataFrame()
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

    # default params (override if provided)
    if params is None:
        params = {
            "objective": "binary",
            "boosting_type": "gbdt",
            "metric": "auc",
            "learning_rate": 0.05,
            "num_leaves": 31,
            "max_depth": -1,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "min_child_samples": 20,
            "verbosity": -1,
            "seed": seed
        }

    for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {fold+1}/{n_splits}")
        X_tr, y_tr = X.iloc[tr_idx].reset_index(drop=True), y[tr_idx]
        X_val, y_val = X.iloc[val_idx].reset_index(drop=True), y[val_idx]

        dtrain = lgb.Dataset(X_tr, label=y_tr)
        dval   = lgb.Dataset(X_val, label=y_val)

        clf = lgb.train(
            params,
            dtrain,
            num_boost_round=5000,
            valid_sets=[dtrain, dval],
            valid_names=["train", "valid"],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100),
                lgb.log_evaluation(period=200)
            ]
        )

        best_it = clf.best_iteration
        print(f" best_iteration: {best_it}")

        oof_preds[val_idx] = clf.predict(X_val, num_iteration=best_it)
        test_preds += clf.predict(X_test[features_list], num_iteration=best_it) / n_splits

        fold_imp = pd.DataFrame({
            "feature": features_list,
            "importance": clf.feature_importance(importance_type="gain"),
            "fold": fold + 1
        })
        feature_importance_df = pd.concat([feature_importance_df, fold_imp], axis=0, ignore_index=True)

        fold_auc = roc_auc_score(y_val, oof_preds[val_idx])
        print(f" Fold {fold+1} AUC: {fold_auc:.6f}\n")
        gc.collect()

    cv_auc = roc_auc_score(y, oof_preds)
    print("CV AUC:", cv_auc)

    return oof_preds, test_preds, feature_importance_df

# -----------------------
# 4) Diagnostics Agent
# -----------------------
def diagnostics_agent(X: pd.DataFrame, y: np.ndarray, oof: np.ndarray, fi: pd.DataFrame, top_k: int = 15) -> Dict[str, Any]:
    report = {}
    report["auc"] = float(roc_auc_score(y, oof))
    diffs = np.abs(oof - y)
    hard_idx = np.argsort(diffs)[-5000:] if len(diffs) > 5000 else np.argsort(diffs)
    report["hard_samples"] = hard_idx.tolist()
    top_feat = fi.groupby("feature")["importance"].mean().sort_values(ascending=False).head(top_k)
    report["top_features"] = top_feat.index.tolist()
    drift_scores = {}
    for col in X.columns:
        try:
            drift_scores[col] = abs(train[col].mean() - test[col].mean())
        except Exception:
            drift_scores[col] = 0.0
    drift_sorted = sorted(drift_scores.items(), key=lambda x: x[1], reverse=True)
    report["drift_features"] = [c for c, _ in drift_sorted[:10]]
    report["oof_minmax"] = (float(oof.min()), float(oof.max()))
    report["positive_ratio"] = float(y.mean())
    return report

# -----------------------
# 5) Strategy Agent
# -----------------------
def strategy_agent(report: Dict[str, Any]) -> List[str]:
    strategies = []
    if report["auc"] < 0.90:
        strategies.append("try_catboost")
        strategies.append("add_interactions")
    if len(report.get("drift_features", [])) > 0:
        strategies.append("add_drift_ratio_features")
    strategies.append("add_poly_top_features")
    strategies.append("tune_hyperparams")
    return strategies

# -----------------------
# 6) Execution Agent
# -----------------------
def execution_agent(strategies: List[str],
                    X_train: pd.DataFrame,
                    X_test: pd.DataFrame,
                    features_list: List[str],
                    max_interactions: int = 20) -> Tuple[pd.DataFrame, pd.DataFrame, List[str]]:
    new_feats = []
    X_tr = X_train.copy()
    X_te = X_test.copy()

    if "add_interactions" in strategies:
        numeric_cols = [c for c in features_list if X_tr[c].dtype != "object"]
        top_num = numeric_cols[:10]
        cnt = 0
        for i, f1 in enumerate(top_num):
            for j, f2 in enumerate(top_num):
                if j <= i:
                    continue
                name = f"{f1}_x_{f2}"
                X_tr[name] = X_tr[f1] * X_tr[f2]
                X_te[name] = X_te[f1] * X_te[f2]
                new_feats.append(name)
                cnt += 1
                if cnt >= max_interactions:
                    break
            if cnt >= max_interactions:
                break

    if "add_drift_ratio_features" in strategies:
        for col in (features_list[:20]):
            name = f"{col}_drift_ratio"
            denom = X_tr[col].mean() + 1e-9
            X_tr[name] = X_tr[col] / denom
            X_te[name] = X_te[col] / denom
            new_feats.append(name)

    if "add_poly_top_features" in strategies:
        for col in features_list[:8]:
            name = f"{col}_sq"
            X_tr[name] = X_tr[col] ** 2
            X_te[name] = X_te[col] ** 2
            new_feats.append(name)

    X_tr.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_te.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_tr = X_tr.fillna(X_tr.median())
    X_te = X_te.fillna(X_te.median())

    features_new = features_list + new_feats
    return X_tr, X_te, features_new

# -----------------------
# 7) Hyperparameter tuner (lightweight)
# -----------------------
def get_tuned_params(base_params: dict, iteration: int = 0) -> dict:
    params = base_params.copy()
    if iteration % 3 == 0:
        params["num_leaves"] = 31 + 8 * (iteration % 4)
        params["learning_rate"] = 0.05 * (0.8 ** (iteration % 3))
    elif iteration % 3 == 1:
        params["num_leaves"] = 40 + 6 * (iteration % 4)
        params["learning_rate"] = 0.03 * (0.9 ** (iteration % 3))
    else:
        params["num_leaves"] = 24 + 12 * (iteration % 4)
        params["learning_rate"] = 0.08 * (0.85 ** (iteration % 3))
    params["reg_alpha"] = 0.0 + 0.5 * (iteration % 2)
    params["reg_lambda"] = 0.0 + 0.5 * ((iteration+1) % 2)
    return params

# -----------------------
# 8) Agentic Orchestrator (main loop) with early stop on AUC
# -----------------------
def agentic_training_loop(max_rounds: int = 4, stop_auc: float = 0.72):
    X_current = train[features].copy()
    X_test_current = test[features].copy()
    feature_list = features.copy()
    best_auc_overall = 0.0
    memory = []

    base_params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "metric": "auc",
        "learning_rate": 0.05,
        "num_leaves": 31,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "min_child_samples": 20,
        "verbosity": -1,
        "seed": RANDOM_STATE
    }

    for iteration in range(max_rounds):
        print(f"\n========== AGENT ITERATION {iteration+1}/{max_rounds} ==========")
        params = get_tuned_params(base_params, iteration)
        oof_preds, test_preds, fi = train_model_and_return_outputs(
            X=X_current,
            y=train[TARGET].values,
            X_test=X_test_current,
            features_list=feature_list,
            params=params,
            n_splits=N_SPLITS,
            seed=RANDOM_STATE + iteration
        )

        report = diagnostics_agent(X_current, train[TARGET].values, oof_preds, fi)
        print("DIAGNOSTICS:", json.dumps(report, indent=2, default=str))

        if report["auc"] >= stop_auc:
            print(f"Target AUC reached ({report['auc']:.4f} >= {stop_auc}). Stopping.")
            best_auc_overall = max(best_auc_overall, report["auc"])
            memory.append({"iter": iteration+1, "strategies": [], "auc_before": report["auc"], "stopped": True})
            break

        strategies = strategy_agent(report)
        print("STRATEGIES:", strategies)

        X_new, X_test_new, feature_list_new = execution_agent(strategies, X_current, X_test_current, feature_list)
        added = len(feature_list_new) - len(feature_list)
        print(f" New features added: {added}")

        oof_new, test_new, fi_new = train_model_and_return_outputs(
            X=X_new,
            y=train[TARGET].values,
            X_test=X_test_new,
            features_list=feature_list_new,
            params=params,
            n_splits=N_SPLITS,
            seed=RANDOM_STATE + iteration + 100
        )

        auc_before = report["auc"]
        auc_after = float(roc_auc_score(train[TARGET].values, oof_new))
        print(f"AUC before: {auc_before:.6f}  |  AUC after: {auc_after:.6f}")

        if auc_after > auc_before + 1e-6:
            print(" Improvement accepted — updating dataset.")
            X_current = X_new
            X_test_current = X_test_new
            feature_list = feature_list_new
            best_auc_overall = max(best_auc_overall, auc_after)
            mem = {"iter": iteration+1, "strategies": strategies, "auc_before": auc_before, "auc_after": auc_after, "added_features": added}
            memory.append(mem)
            try:
                fi_new_mean = fi_new.groupby("feature")["importance"].mean().sort_values(ascending=False).reset_index()
                fi_new_mean.to_csv(f"feature_importance_iter{iteration+1}.csv", index=False)
                pd.DataFrame({"oof": oof_new}).to_csv(f"oof_iter{iteration+1}.csv", index=False)
            except Exception as e:
                print("Warning saving artifacts:", e)
        else:
            print(" No improvement — reverting to previous feature set.")
            memory.append({"iter": iteration+1, "strategies": strategies, "auc_before": auc_before, "auc_after": auc_after, "reverted": True})

        if best_auc_overall >= stop_auc:
            print(f"Best AUC reached target ({best_auc_overall:.4f} >= {stop_auc}). Stopping.")
            break

        gc.collect()

    print("\n========== FINAL TRAIN ==========")
    final_oof, final_test_preds, final_fi = train_model_and_return_outputs(
        X=X_current,
        y=train[TARGET].values,
        X_test=X_test_current,
        features_list=feature_list,
        params=get_tuned_params(base_params, 99),
        n_splits=N_SPLITS,
        seed=RANDOM_STATE + 999
    )

    final_auc = float(roc_auc_score(train[TARGET].values, final_oof))
    print("Final CV AUC:", final_auc)

    try:
        fi_mean = final_fi.groupby("feature")["importance"].mean().sort_values(ascending=False).reset_index()
        fi_mean.to_csv("feature_importance_final.csv", index=False)
        pd.DataFrame({"oof": final_oof}).to_csv("oof_final.csv", index=False)
    except Exception as e:
        print("Warning saving final artifacts:", e)

    submission = pd.DataFrame({ID: test[ID].values, TARGET: final_test_preds})
    submission[TARGET] = submission[TARGET].clip(0, 1)
    submission.to_csv("submission.csv", index=False)
    print("Saved submission.csv")

    return {
        "final_auc": float(final_auc),
        "best_auc_overall": float(best_auc_overall),
        "memory": memory,
        "features_final": feature_list
    }

# -----------------------
# 9) Run the agentic loop
# -----------------------
if __name__ == "__main__":
    results = agentic_training_loop(max_rounds=3, stop_auc=0.72)
    print("Agentic results summary:", json.dumps(results, indent=2, default=str))




train shape: (700000, 26)
test shape:  (700000, 26)
24 features detected.
Categorical cols: ['gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status']
Target positive ratio: 0.6232957142857143

========== AGENT ITERATION 1/3 ==========
Fold 1/5
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.726803	valid's auc: 0.723631
[400]	train's auc: 0.734112	valid's auc: 0.725853
[600]	train's auc: 0.739849	valid's auc: 0.726641
[800]	train's auc: 0.74513	valid's auc: 0.72706
[1000]	train's auc: 0.750114	valid's auc: 0.727552
[1200]	train's auc: 0.754761	valid's auc: 0.727675
Early stopping, best iteration is:
[1278]	train's auc: 0.756544	valid's auc: 0.727733
 best_iteration: 1278
 Fold 1 AUC: 0.727733

Fold 2/5
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.727199	valid's auc: 0.721198
[400]	train's auc: 0.734362	valid's auc: 0.723369
[600]	train's auc: 0.740281	valid's auc: 0.72455
[80